# Clasificación de Imágenes MRI con SVM + HOG

Este notebook implementa un pipeline clásico de visión computacional usando:

1. Extracción de características HOG (Histogram of Oriented Gradients)
2. Clasificación con SVM (Support Vector Machine)
3. Evaluación de resultados

In [ ]:
import os
import cv2
import numpy as np
from sklearn.svm import SVC
from skimage.feature import hog
from sklearn.metrics import classification_report

# Configuración de rutas
DATASET_DIR = "../total/archive/Training"

In [ ]:
# Ejemplo de extracción de HOG y entrenamiento SVM
# features, labels = [], []
# for img_path in ...:
#     img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
#     hog_feat = hog(img, pixels_per_cell=(16, 16), cells_per_block=(2, 2))
#     features.append(hog_feat)
#     labels.append(label)
# clf = SVC(kernel='linear')
# clf.fit(features, labels)

Este pipeline es útil como línea base y para comparar con modelos de deep learning.

# Clasificación de Tumores Cerebrales con SVM + HOG

Este notebook implementa un pipeline clásico de visión computacional usando:
- Extracción de características HOG (Histogram of Oriented Gradients)
- Clasificación con SVM (Support Vector Machine)
- Evaluación de resultados

In [ ]:
import os
import cv2
import numpy as np
from pathlib import Path
from sklearn.svm import SVC
from skimage.feature import hog
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# Parámetros
IMG_SIZE = 128  # Tamaño reducido para HOG
BASE_DIR = Path(os.path.join(os.getcwd(), '../total/archive/Training'))

# Cargar imágenes y etiquetas
X, y = [], []
class_names = sorted([d.name for d in BASE_DIR.iterdir() if d.is_dir()])
for idx, class_name in enumerate(class_names):
    class_dir = BASE_DIR / class_name
    for img_path in class_dir.glob('*.jpg'):
        img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        features = hog(img, pixels_per_cell=(16, 16), cells_per_block=(2, 2), feature_vector=True)
        X.append(features)
        y.append(idx)
X = np.array(X)
y = np.array(y)

# Separar en entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# Entrenar SVM
clf = SVC(kernel='linear', probability=True)
clf.fit(X_train, y_train)

# Evaluar
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=class_names))
print('Matriz de confusión:')
print(confusion_matrix(y_test, y_pred))

**Nota:** Este pipeline es útil como línea base y para comparar con modelos de deep learning.